# 加载模型并画图

In [17]:
import os
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd

from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.calibration import CalibratedClassifierCV

from skopt.space import Real, Categorical
from skopt import BayesSearchCV
from sklearn.model_selection import cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_curve, confusion_matrix, precision_score, accuracy_score, f1_score, matthews_corrcoef, auc

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

In [18]:
import matplotlib as mpl

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.use14corefonts'] = False
# mpl.rcParams['pdf.usecorefonts'] = True
mpl.rcParams['pdf.compression'] = 9

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'nature'])

In [19]:
model_name = ["LogisticRegression"][0]

a = 1
f = pd.read_excel(f'/mnt/md0/Public/T3_T4/feature.xlsx')
feature = f.iloc[0:,1:-1]
feature_ = feature.astype("float").values
target = f.loc[0:,'拔管成功']

target_ = target.values
# auc PDF的输出
plot_output_pdf_general_path = f"5FV/2_5FV/{a}"

定义常用变量

In [20]:
predict_result_list_30 = []

for Kfold_id, (train_id, test_id) in enumerate(
    StratifiedKFold(
        n_splits=5, 
        shuffle=True, 
        random_state=42
    ).split(feature_, target_)
):
    test_feature_ = feature_[test_id]
    test_target_ = target_[test_id]

    model = pickle.load(
        open(f"5FV/2_5FV/{a}/{Kfold_id}/{model_name}.pkl", "br")
    )

    predict_result_list_30.append([
        model.predict_proba(X=test_feature_)[:, 1],
        test_target_
    ])


In [21]:
def plot_roc_curve(
    list1,
    path_to_: str
):
    plt.figure(figsize=(19.2 / 4, 10.8 / 4))
    plt.axis('square')

    
    base_fpr = np.linspace(0, 1, 101)
    
    colors = ["red"]  # 定义颜色列表，可以根据需要进行扩展

    for i, predict_result_list in enumerate(list1):
        color = colors[i % len(colors)] 
        tprs = []
        
        for item in predict_result_list:
            target, pred = item[1], item[0]
            fpr, tpr, thresholds = roc_curve(target, pred)
            roc_auc = auc(fpr, tpr)
            
            tpr = np.interp(base_fpr, fpr, tpr)
            tpr[0] = 0.0
            tprs.append(tpr)
            
        tprs = np.array(tprs)
        mean_tprs = tprs.mean(axis=0)
        std = tprs.std(axis=0)
    
        plt.plot(
                base_fpr, mean_tprs, color=color, lw=1,
                
                label= f'LR ROC = %0.2f' % auc(base_fpr, mean_tprs)
            )
            
        

    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) curve')
    plt.legend(loc="lower right")

    plt.savefig(f"{path_to_}", transparent=True)
    plt.clf()

In [22]:
plot_roc_curve(
    [ predict_result_list_30],
    path_to_=f"{plot_output_pdf_general_path}5FCV_roc.pdf"
)

<Figure size 480x270 with 0 Axes>